In [1]:
import pandas as pd
import numpy as np
import io
from helper import *

In [2]:
# Start with the draft data
allDraftData = pd.read_csv("DraftPlacing.csv")
draftData = []
year = 2019
for i in range(len(allDraftData)):
    temp = allDraftData["Text"].iloc[i]
    if type(temp)==float: 
        continue
    temp = pd.read_csv(io.StringIO(temp[80:]), lineterminator='\n')
    temp.columns = temp.iloc[0]
    temp.drop([0], axis=0, inplace=True)
    temp = temp[pd.isnull(temp["Player"])==False]
    # Fix the player name
    temp["Player"] = formatNames([name.split("\\")[0] for name in temp["Player"]])
    temp["Year"] = year
    year -= 1
    draftData.append(temp[["Pk", "Tm", "Player", "Year"]])
    
draftData = pd.concat(draftData, ignore_index=True)
draftData = draftData[draftData["Year"]<2018]
draftData = draftData[draftData["Year"]>1999]

In [3]:
draftData

,Pk,Tm,Player,Year
120,1,PHI,markellefultz,2017
121,2,LAL,lonzoball,2017
122,3,BOS,jaysontatum,2017
123,4,PHO,joshjackson,2017
124,5,SAC,deaaronfox,2017
...,...,...,...,...
1184,54,SAS,coryhightower,2000
1185,55,GSW,chrisporter,2000
1186,56,IND,jaquaywalls,2000
1187,57,ATL,scooniepenn,2000


In [5]:
# Now we will append combine data!
combineData = pd.read_csv("CombineData.csv")
combineData["PLAYER"] = formatNames(combineData["PLAYER"])
# Rename hand stuff
combineData.rename(columns={"HAND L":"HAND_LENGTH", "HAND W":"HAND_WIDTH", "WING DIFF":"WING_DIFF"}, inplace=True)
# Add wing diff
draftData = draftData.merge(combineData, how='left', left_on="Player", right_on="PLAYER").drop(["PLAYER"], axis=1)

In [6]:
draftData

,Pk,Tm,Player,Year,YEAR,POS,HEIGHT,HEIGHT_W_SHOES,REACH,WEIGHT,WINGSPAN,WING_DIFF,SHUTTLE_RUN,THREE_QUARTER_SPRINT,STANDING_VERTICAL,MAX_VERTICAL,BENCH_PRESS,BODY_FAT,HAND_LENGTH,HAND_WIDTH
0,1,PHI,markellefultz,2017,2017.0,PG,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,LAL,lonzoball,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,BOS,jaysontatum,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,PHO,joshjackson,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,SAC,deaaronfox,2017,2017.0,PG,74.00,75.25,100.0,169.6,78.5,4.50,NaN,NaN,NaN,NaN,NaN,4.50%,8.5,8.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,54,SAS,coryhightower,2000,2000.0,WING,77.50,NaN,101.5,183.0,83.0,5.50,NaN,3.37,29.5,34.5,0,NaN,NaN,NaN
1066,55,GSW,chrisporter,2000,2000.0,COMBO F,77.50,NaN,103.5,214.0,81.5,4.00,NaN,3.20,28.5,35.0,6,NaN,NaN,NaN
1067,56,IND,jaquaywalls,2000,2000.0,PG,73.25,NaN,96.5,165.5,74.5,1.25,NaN,3.23,28.5,35.5,0,NaN,NaN,NaN
1068,57,ATL,scooniepenn,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Finally we tackle the player data
playerData1 = pd.read_csv("allPlayerData.csv")
playerData2 = pd.read_csv("PLAYERDATA.csv")

playerData1.drop(["Root"], axis=1, inplace=True)
playerData2.drop(["Root"], axis=1, inplace=True)

playerData1.rename(columns={"Text1":"Player", "Text":"perGame", "Text2":"totals", "Text3":"per40min", 
                            "Text4":"per100pos", "Text5":"advanced", "Text6":"awards"}, inplace=True)
playerData2.rename(columns={"Text":"Player", "Text1":"perGame", "Text2":"totals", "Text3":"per40min", 
                           "Text4":"per100pos", "Text5":"advanced", "Text6":"awards"}, inplace=True)
playerData1["Player"] = formatNames(playerData1["Player"])
playerData2["Player"] = formatNames(playerData2["Player"])
# Combine the two
playerData = pd.DataFrame(columns=playerData1.columns)
for name in np.unique(playerData1["Player"].to_list() + playerData2["Player"].to_list()):
    p1 = playerData1[playerData1['Player']==name]
    p2 = playerData2[playerData2['Player']==name]
    
    if p1.empty:
        playerData = playerData.append(p2, ignore_index=True)
    else:
        playerData = playerData.append(p1, ignore_index=True)

playerData.drop_duplicates(subset=["Player"], inplace=True, keep="first")
playerData = playerData[pd.isnull(playerData["Player"])==False]

perGameCols = ["gamesPlayed", "minutes", "FT%", "3P%", "SOS"]
advancedCols = ["PER", "eFG%", "ORB%", "DRB%", "AST%", "TOV%", "STL%", "BLK%", "USG%", "OWS", "DWS"]
totalCols = ["FTA", "FGA", "MP", "3PA", "PTS", "FGA", "PF", "AST", "TOV", "G"]
posCols = ["ORtg", "DRtg"]
calculatedCols = ["MP_per_PF", "FTA_per_FGA", "MP_per_3PA", "PTS_per_FGA", "AST_per_TOV", "PPG", "PPM"]
awardsCol = ["awards"]
allColumns = perGameCols + advancedCols + totalCols + posCols + calculatedCols

playerDF = pd.DataFrame(index=range(len(playerData)))
playerDF['Player'] = np.nan
playerDF["collegeYear"] = np.nan
for col in allColumns: 
    playerDF[col] = np.nan
playerDF["awards"] = 0

for i in range(len(playerData)):
    player = playerData.iloc[i]
    playerDF["Player"].iloc[i] = player["Player"]
    # Get the perGame data
    perGame = player["perGame"]
    if type(perGame)!=float: 
        perGame = pd.read_csv(io.StringIO(perGame[80:]), lineterminator='\n')
        perGame = perGame.iloc[perGame[perGame["Season"]=="Career"].index[0]-1, :]
        perGame.rename({"G":"gamesPlayed", "MP":"minutes"}, inplace=True)
        playerDF.loc[i, "collegeYear"] = perGame["Season"]
        for col in perGameCols:
            if col in perGame.index:
                playerDF.loc[i, col] = perGame[col]
            
    # Get the advanced data
    advanced = player["advanced"]
    if type(advanced)!=float:
        advanced = pd.read_csv(io.StringIO(advanced[80:]), lineterminator='\n')
        advanced = advanced.iloc[advanced[advanced["Season"]=="Career"].index[0]-1, :]
        playerDF.loc[i, "collegeYear"] = advanced["Season"]
        
        for col in advancedCols:
            if col in advanced.index:
                playerDF.loc[i, col] = advanced[col]
            
    # Get totals data
    totals = player["totals"]
    if type(totals)!=float:
        totals = pd.read_csv(io.StringIO(totals[80:]), lineterminator='\n')
        totals = totals.iloc[totals[totals["Season"]=="Career"].index[0]-1, :]
        playerDF.loc[i, "collegeYear"] = totals["Season"]
        
        for col in totalCols:
            if col != "G":
                if col in totals.index: 
                    playerDF.loc[i, col] = totals[col]
    
        # Calculate the derrived metrics
        if "MP" and "PF" in totals.index:
            playerDF.loc[i, "MP_per_PF"] = totals["MP"]/totals["PF"] if totals["PF"]!=0 else np.nan # SHOULD BE FIXED
        if "FTA" and "FGA" in totals.index:    
            playerDF.loc[i, "FTA_per_FGA"] = totals["FTA"]/totals["FGA"] # ASSUMED NOT ZERO
        if "MP" and "3PA" in totals.index:
            playerDF.loc[i, "MP_per_3PA"] = totals["MP"]/totals["3PA"] if totals["3PA"]!=0 else np.nan # SHOULD BE FIXED
        if "PTS" and "FGA" in totals.index:
            playerDF.loc[i, "PTS_per_FGA"] = totals["PTS"]/totals["FGA"]
        if "AST" and "TOV" in totals.index:
            playerDF.loc[i, 'AST_per_TOV'] = totals["AST"]/totals["TOV"]
        playerDF.loc[i, 'PPG'] = totals["PTS"]/totals['G']
        playerDF.loc[i, "PPM"] = totals["PTS"]/totals["MP"]
    
    # Get posessions data
    pos = player["per100pos"]
    if type(pos)!=float:
        pos = pd.read_csv(io.StringIO(pos[80:]), lineterminator='\n')
        pos = pos.iloc[pos[pos["Season"]=="Career"].index[0]-1, :]
        playerDF.loc[i, "collegeYear"] = pos["Season"]
        
        for col in posCols:
            if col in pos.index:
                playerDF.loc[i, col] = pos[col]
            
    # Get awards data
    awards = player["awards"]
    if type(awards)!=float:
        playerDF["awards"].iloc[i] = awards.count("20")
        
    if i%100==0:
        print(i/len(playerData))

playerDF.drop(["G"], inplace=True, axis=1)
draftData = draftData.merge(playerDF, how='left', left_on="Player", right_on="Player")

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


0.0
0.06289308176100629
0.12578616352201258
0.18867924528301888
0.25157232704402516
0.31446540880503143
0.37735849056603776
0.44025157232704404
0.5031446540880503
0.5660377358490566
0.6289308176100629
0.6918238993710691
0.7547169811320755
0.8176100628930818
0.8805031446540881
0.9433962264150944


In [5]:
print("LEN:", len(draftData))
for col in draftData:
    print(col, draftData[col].isna().sum())

LEN: 1070
Pk 0
Tm 0
Player 0
Year 0
YEAR 375
POS 375
HEIGHT 379
HEIGHT_W_SHOES 512
REACH 379
WEIGHT 380
WINGSPAN 378
WING_DIFF 376
SHUTTLE_RUN 910
THREE_QUARTER_SPRINT 454
STANDING_VERTICAL 449
MAX_VERTICAL 450
BENCH_PRESS 550
BODY_FAT 477
HAND_LENGTH 731
HAND_WIDTH 731
collegeYear 375
gamesPlayed 379
minutes 379
FT% 379
3P% 445
SOS 379
PER 765
eFG% 763
ORB% 765
DRB% 765
AST% 765
TOV% 765
STL% 765
BLK% 765
USG% 765
OWS 765
DWS 765
FTA 375
FGA 375
MP 375
3PA 375
PTS 375
PF 375
AST 375
TOV 375
ORtg 765
DRtg 765
MP_per_PF 375
FTA_per_FGA 375
MP_per_3PA 441
PTS_per_FGA 375
AST_per_TOV 375
PPG 375
PPM 375
awards 217


In [6]:
# Now we get RSCI and mock draft data
RSCI = pd.read_csv("RSCI_data.csv")
RSCI = RSCI[pd.isnull(RSCI["Name_URL"])==False]
RSCI["Player"] = formatNames([RSCI["Name_URL"].iloc[i].replace("(college)", "") for i in range(len(RSCI))])
RSCI.drop_duplicates(subset=["Player"], inplace=True, keep="first")
RSCICols = ["Player", "RSCI", "m1", "m2", "m3", "m4", "m5", "m6"]

RSCI = RSCI[RSCICols]

draftData = draftData.merge(RSCI, how='left', left_on="Player", right_on="Player")

In [7]:
# Look at missing values
print("LEN:", len(draftData))
for col in draftData:
    print(col, draftData[col].isna().sum())

LEN: 1070
Pk 0
Tm 0
Player 0
Year 0
YEAR 375
POS 375
HEIGHT 379
HEIGHT_W_SHOES 512
REACH 379
WEIGHT 380
WINGSPAN 378
WING_DIFF 376
SHUTTLE_RUN 910
THREE_QUARTER_SPRINT 454
STANDING_VERTICAL 449
MAX_VERTICAL 450
BENCH_PRESS 550
BODY_FAT 477
HAND_LENGTH 731
HAND_WIDTH 731
collegeYear 375
gamesPlayed 379
minutes 379
FT% 379
3P% 445
SOS 379
PER 765
eFG% 763
ORB% 765
DRB% 765
AST% 765
TOV% 765
STL% 765
BLK% 765
USG% 765
OWS 765
DWS 765
FTA 375
FGA 375
MP 375
3PA 375
PTS 375
PF 375
AST 375
TOV 375
ORtg 765
DRtg 765
MP_per_PF 375
FTA_per_FGA 375
MP_per_3PA 441
PTS_per_FGA 375
AST_per_TOV 375
PPG 375
PPM 375
awards 217
RSCI 544
m1 600
m2 588
m3 572
m4 567
m5 644
m6 726


In [8]:
# Look at position
draftData["POS"] = [s.replace("COMBO ", "").replace("WING", "F").replace("PG", "G").replace("BIG", "C") if type(s)==str else s for s in draftData["POS"]]

# Save it
draftData.to_pickle("featureData.df")

In [9]:
draftData["POS"].unique()

array(['G', nan, 'C', 'F'], dtype=object)

In [10]:
draftData.columns

Index(['Pk', 'Tm', 'Player', 'Year', 'YEAR', 'POS', 'HEIGHT', 'HEIGHT_W_SHOES',
       'REACH', 'WEIGHT', 'WINGSPAN', 'WING_DIFF', 'SHUTTLE_RUN',
       'THREE_QUARTER_SPRINT', 'STANDING_VERTICAL', 'MAX_VERTICAL',
       'BENCH_PRESS', 'BODY_FAT', 'HAND_LENGTH', 'HAND_WIDTH', 'collegeYear',
       'gamesPlayed', 'minutes', 'FT%', '3P%', 'SOS', 'PER', 'eFG%', 'ORB%',
       'DRB%', 'AST%', 'TOV%', 'STL%', 'BLK%', 'USG%', 'OWS', 'DWS', 'FTA',
       'FGA', 'MP', '3PA', 'PTS', 'PF', 'AST', 'TOV', 'ORtg', 'DRtg',
       'MP_per_PF', 'FTA_per_FGA', 'MP_per_3PA', 'PTS_per_FGA', 'AST_per_TOV',
       'PPG', 'PPM', 'awards', 'RSCI', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6'],
      dtype='object')